In [1]:
import sys
sys.path.append('../')
from utils import *

In [2]:
weekday_dict = {0:'Mon', 1:'Tues', 2:'Wed', 3:'Thur', 4:'Fri', 5:'Sat', 6:'Sun'}
month_dict = {1:'Jan', 2:'Feb', 3:'Mar', 4:'April'}
def clean_df(df):
    df.columns = [c.lower().strip().replace(" ", "_") for c in df.columns]
    return df

In [3]:
# Get all filenames
province_path = data_path + 'midas_china/daily_cases_chinacdc_EN/'
fnames = os.listdir(province_path)
fnames = [f for f in fnames if f not in ['data_guide.csv', 'collection_metadata.csv']]
print(len(fnames))

86


In [4]:
# Create chinese_all
col_names = clean_df(pd.read_csv(province_path+fnames[0])).columns
df_all = pd.DataFrame(columns=col_names)

for f in fnames:
    df = pd.read_csv(province_path+f)
    df = clean_df(df)
    df['date'] = f[:10]
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].apply(lambda x: x.month).map(month_dict)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['weekday'] = df['date'].apply(lambda x: x.weekday()).map(weekday_dict)
    df['date'] = df['date'].astype(str)
    df_all = df_all.append(df, sort=False)
    
# Save chinese_all.csv
data_cols = ['add_suspect', 'cumulative_suspect', 'new_diagnosis', 'cumulative_diagnosis', 'added_death', 'cumulative_death']
for c in data_cols:
    df_all[c] = df_all[c].astype(int)

print(df_all.shape)
df_all.to_csv(data_path+'midas_china_mst/chinese_all.csv', index=None)

(2924, 11)


In [5]:
# Build and save chinese_by_date
df_date = df_all.groupby('date').sum().reset_index().drop(['day'],1)
df_date = pd.merge(df_date, df_all[['date', 'month','day','weekday']].drop_duplicates(), on = 'date', how = 'inner')
print(df_date.shape)
df_date.to_csv(data_path+'midas_china_mst/chinese_by_date.csv', index=None)

(86, 10)


In [ ]:
# Build and save province by_province, by_date
def create_unique_files_for_col(df, colname):
    save_path = data_path+'midas_china_mst/by_' + colname + '/'
    if not os.path.exists(save_path): 
        os.system('mkdir '+ save_path)
        print("Creating path:", save_path)
    unique_vals = list(df[colname].unique())
    for v in unique_vals:
        temp = df[df[colname] == v]
        temp = temp.sort_values('date').reset_index(drop=True)
        fname = v.strip().lower().replace(" ","_")
        temp.to_csv(save_path+fname+'.csv',index=None)
    print(f"Number of files in by_{colname}:", len(os.listdir(save_path)))
    
create_unique_files_for_col(df_all, 'date')
create_unique_files_for_col(df_all, 'province')